#AutoEncoders

##Downloading the dataset

###ML-100K

In [ ]:
!wget "http://files.grouplens.org/datasets/movielens/ml-100k.zip"
!unzip ml-100k.zip
!ls

In [2]:
!wget "http://files.grouplens.org/datasets/movielens/ml-1m.zip"
!unzip ml-1m.zip
!ls

zsh:1: command not found: wget
Archive:  ml-1m.zip
   creating: ml-1m/
  inflating: ml-1m/.DS_Store         
   creating: __MACOSX/ml-1m/
  inflating: __MACOSX/ml-1m/._.DS_Store  
  inflating: ml-1m/.Rhistory         
  inflating: ml-1m/movies.dat        
  inflating: __MACOSX/ml-1m/._movies.dat  
  inflating: ml-1m/ratings.dat       
  inflating: __MACOSX/ml-1m/._ratings.dat  
  inflating: ml-1m/README            
  inflating: __MACOSX/ml-1m/._README  
  inflating: ml-1m/test_set.csv      
  inflating: __MACOSX/ml-1m/._test_set.csv  
  inflating: ml-1m/training_set.csv  
  inflating: __MACOSX/ml-1m/._training_set.csv  
  inflating: ml-1m/users.dat         
  inflating: __MACOSX/ml-1m/._users.dat  
  inflating: __MACOSX/._ml-1m        
AutoEncoders.ipynb autoencoders.py    ml-100k.zip        ml-1m.zip
__MACOSX           ml-100k            ml-1m


##Importing the libraries

In [3]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

## Importing the dataset


In [41]:
# We won't be using this dataset.
movies = pd.read_csv('ml-1m/movies.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
users = pd.read_csv('ml-1m/users.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
ratings = pd.read_csv('ml-1m/ratings.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')

In [42]:
movies

,0,1,2
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama


In [43]:
users

,0,1,2,3,4
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455
...,...,...,...,...,...
6035,6036,F,25,15,32603
6036,6037,F,45,1,76006
6037,6038,F,56,1,14706
6038,6039,F,45,0,01060


In [44]:
ratings

,0,1,2,3
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


## Preparing the training set and the test set


In [45]:
training_set = pd.read_csv('ml-100k/u1.base', delimiter = '\t')
#training_set = np.array(training_set, dtype = 'int')
test_set = pd.read_csv('ml-100k/u1.test', delimiter = '\t')
#test_set = np.array(test_set, dtype = 'int')

In [46]:
training_set

,1,1.1,5,874965758
0,1,2,3,876893171
1,1,3,4,878542960
2,1,4,3,876893119
3,1,5,3,889751712
4,1,7,4,875071561
...,...,...,...,...
79994,943,1067,2,875501756
79995,943,1074,4,888640250
79996,943,1188,3,888640250
79997,943,1228,3,888640275


In [47]:
test_set

,1,6,5,887431973
0,1,10,3,875693118
1,1,12,5,878542960
2,1,14,5,874965706
3,1,17,3,875073198
4,1,20,4,887431883
...,...,...,...,...
19994,458,648,4,886395899
19995,458,1101,4,886397931
19996,459,934,3,879563639
19997,460,10,3,882912371


In [48]:
training_set = np.array(training_set, dtype = 'int')
test_set = np.array(test_set, dtype = 'int')

In [49]:
training_set

array([[        1,         2,         3, 876893171],
       [        1,         3,         4, 878542960],
       [        1,         4,         3, 876893119],
       ...,
       [      943,      1188,         3, 888640250],
       [      943,      1228,         3, 888640275],
       [      943,      1330,         3, 888692465]])

In [50]:
test_set

array([[        1,        10,         3, 875693118],
       [        1,        12,         5, 878542960],
       [        1,        14,         5, 874965706],
       ...,
       [      459,       934,         3, 879563639],
       [      460,        10,         3, 882912371],
       [      462,       682,         5, 886365231]])

## Getting the number of users and movies


In [51]:
nb_users = int(max(max(training_set[:, 0], ), max(test_set[:, 0])))
nb_movies = int(max(max(training_set[:, 1], ), max(test_set[:, 1])))

In [52]:
nb_users

943

In [53]:
nb_movies

1682

## Converting the data into an array with users in lines and movies in columns


In [54]:
def convert(data):
  new_data = []
  for id_users in range(1, nb_users + 1):
    id_movies = data[:, 1] [data[:, 0] == id_users]
    id_ratings = data[:, 2] [data[:, 0] == id_users]
    ratings = np.zeros(nb_movies)
    ratings[id_movies - 1] = id_ratings
    new_data.append(list(ratings))
  return new_data
training_set = convert(training_set)
test_set = convert(test_set)

## Converting the data into Torch tensors


In [55]:
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

In [61]:
len(training_set)

943

In [62]:
len(training_set[0])

1682

In [57]:
test_set

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

## Creating the architecture of the Neural Network


In [9]:
class SAE(nn.Module):
    def __init__(self, ):
        super(SAE, self).__init__()
        self.fc1 = nn.Linear(nb_movies, 20)
        self.fc2 = nn.Linear(20, 10)
        self.fc3 = nn.Linear(10, 20)
        self.fc4 = nn.Linear(20, nb_movies)
        self.activation = nn.Sigmoid()
    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x)
        return x
sae = SAE()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr = 0.01, weight_decay = 0.5)

## Training the SAE


In [10]:
nb_epoch = 200
for epoch in range(1, nb_epoch + 1):
  train_loss = 0
  s = 0.
  for id_user in range(nb_users):
    input = Variable(training_set[id_user]).unsqueeze(0)
    target = input.clone()
    if torch.sum(target.data > 0) > 0:
      output = sae(input)
      target.require_grad = False
      output[target == 0] = 0
      loss = criterion(output, target)
      mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
      loss.backward()
      train_loss += np.sqrt(loss.data*mean_corrector)
      s += 1.
      optimizer.step()
  print('epoch: '+str(epoch)+'loss: '+ str(train_loss/s))

epoch: 1loss: tensor(1.7708)
epoch: 2loss: tensor(1.0967)
epoch: 3loss: tensor(1.0535)
epoch: 4loss: tensor(1.0382)
epoch: 5loss: tensor(1.0307)
epoch: 6loss: tensor(1.0267)
epoch: 7loss: tensor(1.0238)
epoch: 8loss: tensor(1.0220)
epoch: 9loss: tensor(1.0209)
epoch: 10loss: tensor(1.0198)
epoch: 11loss: tensor(1.0188)
epoch: 12loss: tensor(1.0185)
epoch: 13loss: tensor(1.0181)
epoch: 14loss: tensor(1.0175)
epoch: 15loss: tensor(1.0172)
epoch: 16loss: tensor(1.0171)
epoch: 17loss: tensor(1.0168)
epoch: 18loss: tensor(1.0164)
epoch: 19loss: tensor(1.0162)
epoch: 20loss: tensor(1.0162)
epoch: 21loss: tensor(1.0160)
epoch: 22loss: tensor(1.0161)
epoch: 23loss: tensor(1.0162)
epoch: 24loss: tensor(1.0159)
epoch: 25loss: tensor(1.0156)
epoch: 26loss: tensor(1.0158)
epoch: 27loss: tensor(1.0151)
epoch: 28loss: tensor(1.0154)
epoch: 29loss: tensor(1.0125)
epoch: 30loss: tensor(1.0119)
epoch: 31loss: tensor(1.0105)
epoch: 32loss: tensor(1.0077)
epoch: 33loss: tensor(1.0076)
epoch: 34loss: tens

## Testing the SAE


In [11]:
test_loss = 0
s = 0.
for id_user in range(nb_users):
  input = Variable(training_set[id_user]).unsqueeze(0)
  target = Variable(test_set[id_user]).unsqueeze(0)
  if torch.sum(target.data > 0) > 0:
    output = sae(input)
    target.require_grad = False
    output[target == 0] = 0
    loss = criterion(output, target)
    mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
    test_loss += np.sqrt(loss.data*mean_corrector)
    s += 1.
print('test loss: '+str(test_loss/s))

test loss: tensor(0.9466)
